In [1]:
# These are some commonly used R Packages.  
# The arrow package makes loading data with spark faster. 
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(data.table, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(truveta.notebook.study)
library(sparklyr)
library(ggplot2)
library(reshape2)
library(survival)

In [2]:
print("load snapshot")
con <- create_connection()
study <- get_study(con)
#print(study)
population <- get_population(con, id='p-an3klyy7kz2u7hq3fngqv7v63i')
snapshot <- get_latest_snapshot(con, population)
snapshot

In [3]:
# get list of tables from the snapshot
tables <- get_tables(con, snapshot)

In [4]:
#Get your working directory
# use fs = true when reading and writing files locally

output_path_local <- paste0(get_output_path(con, study, fs = TRUE),"/")
output_path_local

In [44]:
#Write the file
# Build path
file_to_write <- paste(output_path_local, "topDrug.csv", sep = "")

# use write.csv to write your file
write.csv(topDrug, file_to_write, row.names = FALSE)

In [8]:
# Read the file
t1 <- paste(output_path_local, "MED_QC.csv", sep = "")
# use read.csv to read file into a R dataframe
MED_QC <- read.csv(t1)

In [9]:
# Convert R DataFrame to Spark DataFrame
MED_QC_sql <- as.DataFrame(MED_QC)
createOrReplaceTempView(MED_QC_sql, "MED_QC_sql") 

#### Only derive EXQC

In [10]:
# Use this for future run
# Run the med_qc_sql first
# The expstatus & exstdt should be in EXQC 

med_flag <- function(codes = "codes", med_name = "name") {
create_view(codes, "med_codes")
  
sql <- "
 
WITH map as 
(
    SELECT mcm.Id, cc.*
    FROM  MedicationCodeConceptMap mcm 
    JOIN Concept cc on (mcm.CodeConceptId = cc.ConceptId)
    WHERE mcm.CodeConceptId IN
        (SELECT ConceptId FROM med_codes)
),

dis1 as 
(
    SELECT  DISTINCT d.PersonId, d.DispenseDateTime, d.DaysSupply
    FROM MedicationDispense d
    INNER JOIN map mcm
       ON mcm.Id = d.CodeConceptMapId
    WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)
),

dis2 as
(
    SELECT PersonId, min(DispenseDateTime) as DSSTDT, max(DispenseDateTime) as DSENDT,
    count(DISTINCT DispenseDateTime) as TotNumDisRecDrug, sum(DaysSupply) as TotalDaysSupply
    FROM dis1
    GROUP BY PersonId
),

req1 as 
(
    SELECT DISTINCT r.PersonId, r.AuthoredOnDateTime
    FROM MedicationRequest r
    INNER JOIN map mcm
       ON mcm.Id = r.CodeConceptMapId
    WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)
),

req2 as
(
    SELECT PersonId, min(AuthoredOnDateTime) as RQSTDT, max(AuthoredOnDateTime) as RQENDT,
     count(DISTINCT AuthoredOnDateTime) as TotNumReqRecDrug
    FROM req1
    GROUP BY PersonId
),

final as (
    SELECT 
        COALESCE(d.PersonId, r.PersonId) AS PersonId,
        d.DSSTDT, d.DSENDT, COALESCE(d.TotNumDisRecDrug, 0) AS TotNumDisRecDrug, d.TotalDaysSupply, 
        CASE WHEN d.DSSTDT IS NOT NULL AND d.DSENDT IS NOT NULL THEN DATEDIFF(d.DSENDT, d.DSSTDT) ELSE NULL END AS DISSDAYSDRUG,
        r.RQSTDT, r.RQENDT, COALESCE(r.TotNumReqRecDrug, 0) AS TotNumReqRecDrug,
        CASE WHEN r.RQSTDT IS NOT NULL AND r.RQENDT IS NOT NULL THEN DATEDIFF(r.RQENDT, r.RQSTDT) ELSE NULL END AS REQDAYSDRUG,
        COALESCE(d.DSSTDT,r.RQSTDT) AS EXSTDT
    FROM dis2 d
    FULL OUTER JOIN req2 r
        ON d.PersonId = r.PersonId
)

SELECT 
    f.*, 
    '%s' AS DRUGNAME,
    CASE 
        WHEN q.TotNumReqRec = 0 AND f.TotNumDisRecDrug >= 1 THEN 'Potential data error'
        WHEN (f.TotNumDisRecDrug >= 1 AND f.TotalDaysSupply >= 15)  OR f.TotNumDisRecDrug >= 2 THEN 'Exposed'
        WHEN f.TotNumReqRecDrug >= 2 AND q.Ratio < 0.5 AND f.TotNumDisRecDrug < 2 THEN 'Exposed'
        WHEN f.TotNumReqRecDrug = 0  AND f.TotNumDisRecDrug = 0 THEN 'Unexposed'
        WHEN f.TotNumReqRecDrug >= 1 AND f.TotNumDisRecDrug = 0 AND q.Ratio >= 1  THEN 'Unexposed'
        ELSE 'Fill status undetermined'
    END AS FillStatus
FROM final f
LEFT JOIN MED_QC_sql q
    ON f.PersonId = q.PersonId;
"
sql1 <- sprintf(sql,med_name)

tb <- load_sql_table(con,snapshot, query = sql1, view_name = 'tb', output_mode = "sparklyr" )

return(tb)

}

In [11]:
# Function to run the whole process for a given drug name
process_med <- function(drug_name) {
  
  # Get the medication code
  med_codes <- codeset_from_prose(
    con,
    snapshot,
    url = "/definitions/doi",
    variable_name = drug_name
  )
  
  # Create the Medication QC dataset
  med_qc <- med_flag(codes = med_codes, med_name = drug_name)
  
  # Write to file
  file_to_write <- file.path(output_path_local, paste0(drug_name, "QC.csv"))
  write.csv(med_qc, file_to_write, row.names = FALSE)
  
  message(paste("Processed:", drug_name, "→ Saved to", file_to_write))
  
  return(med_qc)
}

In [45]:
drug_list <- c("Statin","Losartan","Hydrochlorothiazide","Gabapentin","Fluticasone","Amoxicillin","Metformin","PPI","Sildenafil","LoopDiuretic",
               "Atenolol", "Dexamethasone", "Albuterol", "GLP", "Semaglutide")
med_results <- lapply(drug_list, process_med)

# temporary Run
drug_list <- c("Quetiapine", "Sertraline", "Warfarin", "Acetaminophen", "Levothyroxine", "Lisinopril", "Amlodipine", "Hydralazine", "Metoprolol", 
               "Levothyroxine", "Lisinopril", "Clopidogrel", "Citalopram")
med_results <- lapply(drug_list, process_med)

# New Run
drug_list <- c("Pioglitazone", "TNF")
med_results <- lapply(drug_list, process_med)

In [12]:
# New Run
drug_list <- c("Pioglitazone", "TNF")
med_results <- lapply(drug_list, process_med)

In [22]:
# Same code as above just seperated 

# Get the medication code
Losartan = codeset_from_prose(con,snapshot, url = "/definitions/doi", variable_name = "Losartan") 

#Create the Medication QC dataset
LosartanQC <- med_flag(codes = Losartan, med_name = "Losartan")

# Write the File
file_to_write <- paste(output_path_local, "/LosartanQC.csv", sep = "")
write.csv(LosartanQC, file_to_write, row.names = FALSE)

### Quality check

In [16]:
# Add all the dataset to be read
read_file <- function(Dataset_name) {
  
  # Build file path
  tb <- file.path(output_path_local, paste0(Dataset_name,".csv"))
  
  # Read CSV and assign it to an object with the given name
  assign(Dataset_name, read.csv(tb), envir = .GlobalEnv)
}

# Example use
#drug_list <- c("AmlodipineQC", "HydralazineQC", "MetoprololQC", "LevothyroxineQC", "LisinoprilQC", "ClopidogrelQC","CitalopramQC")
drug_list <- c("PioglitazoneQC", "TNFQC")
temp <- lapply(drug_list,read_file)

In [58]:
cat("LosartanQC:",table(LosartanQC$FillStatus),"\n")
cat("StatinQC:",table(StatinQC$FillStatus),"\n")
cat("HydrochlorothiazideQC:",table(HydrochlorothiazideQC$FillStatus),"\n")
cat("Gabapentin:",table(GabapentinQC$FillStatus),"\n")
cat("FluticasoneQC:",table(FluticasoneQC$FillStatus),"\n")
cat("AmoxicillinQC:",table(AmoxicillinQC$FillStatus),"\n")
cat("MetforminQC:",table(MetforminQC$FillStatus),"\n")
cat("PPIQC:",table(PPIQC$FillStatus),"\n")
cat("SildenafilQC:",table(SildenafilQC$FillStatus),"\n")
cat("LoopDiureticQC:",table(LoopDiureticQC$FillStatus),"\n")
cat("AtenololQC:",table(AtenololQC$FillStatus),"\n")

In [15]:
cat("DexamethasoneQC:",table(DexamethasoneQC$FillStatus),"\n")
cat("GLPQC:",table(GLPQC$FillStatus),"\n")
cat("AlbuterolQC:",table(AlbuterolQC$FillStatus),"\n")

In [17]:
cat("QuetiapineQC:", table(QuetiapineQC$FillStatus), "\n")
cat("SertralineQC:", table(SertralineQC$FillStatus), "\n")
cat("WarfarinQC:", table(WarfarinQC$FillStatus), "\n")
cat("AcetaminophenQC:", table(AcetaminophenQC$FillStatus), "\n")
cat("LevothyroxineQC:", table(LevothyroxineQC$FillStatus), "\n")
cat("LisinoprilQC:", table(LisinoprilQC$FillStatus), "\n")
cat("SemaglutideQC:", table(SemaglutideQC$FillStatus), "\n")

In [26]:
cat("SemaglutideQC:", table(SemaglutideQC$FillStatus), "\n")

In [17]:
cat("PioglitazoneQC:",table(PioglitazoneQC$FillStatus),"\n")
cat("TNFQC:",table(TNFQC$FillStatus),"\n")

In [20]:
cat("AmlodipineQC:", table(AmlodipineQC$FillStatus), "\n")
cat("HydralazineQC:", table(HydralazineQC$FillStatus), "\n")
cat("MetoprololQC:", table(MetoprololQC$FillStatus), "\n")
cat("LevothyroxineQC:", table(LevothyroxineQC$FillStatus), "\n")
cat("LisinoprilQC:", table(LisinoprilQC$FillStatus), "\n")
cat("ClopidogrelQC:", table(ClopidogrelQC$FillStatus), "\n")
cat("CitalopramQC:", table(CitalopramQC$FillStatus), "\n")

In [16]:
display_df(LosartanEX)

In [44]:
display_df(GabapentinQC[GabapentinQC$FillStatus=="Fill status undetermined",],10)

### Comorbidity

In [37]:
disease_flag <- function(codes = "codes",disease_name = "name"){
    create_view(codes,"concept_code")

sql <- "
   WITH cond1 as 
   (
     SELECT 
        PersonId, 
        min(COALESCE(OnsetDateTime, RecordedDateTime)) AS CondStart
    FROM condition c
    INNER JOIN ConditionCodeConceptMap ccm 
    ON c.CodeConceptMapId = ccm.Id
    INNER JOIN concept_code pcc 
    ON ccm.CodeConceptId = pcc.ConceptId
    WHERE ccm.SourceConceptId IN (2703595, 2703594)
    AND (OnsetDateTime IS NOT NULL OR RecordedDateTime IS NOT NULL) 
    GROUP BY PersonId
  ),
  
  cond2 AS 
  (
    SELECT p.PersonId,CAST(c.CondStart AS DATE) AS CondStart, CAST(p.MCIDt AS DATE) AS MCIDt
    FROM ADSL_sql p
    INNER JOIN cond1 c
     ON p.PersonId = c.PersonId
  )

  SELECT PersonId, 
    CASE 
    WHEN CondStart IS NOT NULL AND CondStart <= DATEADD(day, 30, MCIDt) 
         THEN 1 ELSE 0 
    END AS %s
  FROM cond2
"
  sql1 <- sprintf(sql, disease_name) 

  tb <- load_sql_table(con,snapshot, query = sql1, output_mode = "sparklyr" ) %>% collect()
  return(tb)
}

In [41]:
IHD = codeset_from_prose(con,snapshot, url = "/definitions/comorbidity", variable_name = "IHD")
df_IHD = disease_flag(IHD,"IHD")

Hypertension = codeset_from_prose(con,snapshot, url = "/definitions/comorbidity", variable_name = "Hypertension") 
df_Hypertension = disease_flag(Hypertension,"Hypertension")

Stroke = codeset_from_prose(con,snapshot, url = "/definitions/comorbidity", variable_name = "Stroke") 
df_stroke = disease_flag(Stroke,"Stroke")

Depression = codeset_from_prose(con,snapshot, url = "/definitions/comorbidity", variable_name = "Depression") 
df_Depression = disease_flag(Depression,"Depression")

T2DM = codeset_from_prose(con,snapshot, url = "/definitions/comorbidity", variable_name = "T2DM") 
df_T2DM = disease_flag(T2DM,"T2DM")

In [50]:
cols_to_replace <- c("IHD", "Hypertension", "Stroke", "Depression", "T2DM")

Comorbidity <- ADSL %>% select(PersonId) %>% 
               left_join(df_IHD, by = "PersonId") %>% 
               left_join(df_Hypertension, by = "PersonId") %>% 
               left_join(df_stroke, by = "PersonId") %>% 
               left_join(df_Depression, by = "PersonId") %>% 
               left_join(df_T2DM, by = "PersonId") %>%
               mutate(across(all_of(cols_to_replace), ~ replace_na(as.integer(.), 0))) 

### Regression Analysis

In [18]:
# Read the file
t1 <- paste(output_path_local, "/ADSL.csv.r", sep = "")
# use read.csv to read file into a R dataframe
ADSL <- read.csv(t1)

t5 <- paste(output_path_local, "/FinalPatient.csv", sep = "")
FinalPatient <- read.csv(t5)

t6 <- paste(output_path_local, "Comorbidity.csv", sep = "")
Comorbidity <- read.csv(t6)

# Dataset with PersonId, All analysis variable, and exposure variable from all drugs(manuscript specific) for both new-user and all-user design
t8 <- paste(output_path_local, "AnalysisDataset.csv.r", sep = "")
AnalysisDataset <- read.csv(t8)

### Derive the multivariate logistic regression results

In [20]:
regression <- function(Dataset_name) {
# Build file path
tb <- paste0(output_path_local,Dataset_name,".csv")
MEDQC1 <- read.csv(tb)

# Step 2: Final dataset to run for analysis

AnalysisData1 <- FinalPatient %>% 
  left_join(ADSL %>% select(PersonId, MCIDT, ADDT, ADT, CNSR, AVAL, Sex, AgeAtDiagnosis, Race, Outcome, Ratio), by = "PersonId") %>%
  left_join(MEDQC1 %>% select(PersonId, FillStatus, EXSTDT), by = "PersonId") %>%
  left_join(Comorbidity, by = "PersonId") %>%
  mutate(FillStatus = ifelse(is.na(FillStatus),"Unexposed",FillStatus))

AnalysisData2 <- AnalysisData1 %>%
  filter(
    Outcome != 2,
    FillStatus %in% c("Unexposed", "Exposed")   # remove "Undetermined"
  ) %>%
  mutate(
    NewExp = case_when(                     # New User
      !is.na(EXSTDT) & FillStatus == "Exposed" &
        as.Date(MCIDT) <= as.Date(EXSTDT) &
        as.Date(EXSTDT) <= as.Date(ADT) ~ 1,
      FillStatus == "Unexposed" |
        (!is.na(EXSTDT) & FillStatus == "Exposed" & as.Date(ADT) < as.Date(EXSTDT)) ~ 0,
       !is.na(EXSTDT) & FillStatus == "Exposed" &  as.Date(MCIDT) > as.Date(EXSTDT) ~ 3,
      TRUE ~ 2
    ),
    AllExp = case_when(                     # All User
      FillStatus == "Exposed" & as.Date(EXSTDT) <= as.Date(ADT) & !is.na(EXSTDT) ~ 1,
      FillStatus == "Unexposed" |
        (FillStatus == "Exposed" & as.Date(ADT) < as.Date(EXSTDT)) ~ 0,
      TRUE ~ 2
    )
  )

# NewUExp - New user exposure;
NewUData <- AnalysisData2 %>% filter(NewExp != 3)
lr_result1 <- glm(Outcome ~ NewExp + Sex + AgeAtDiagnosis + Race + IHD + T2DM + Hypertension + Stroke + Depression, data = NewUData, family = binomial(link = "logit"))

# AllUExp - All User exposure
#AllUData <- AnalysisData2 %>% filter(AllExp != 2)
lr_result2 <- glm(Outcome ~ AllExp + Sex + AgeAtDiagnosis + Race + IHD + T2DM + Hypertension + Stroke + Depression, data = AnalysisData2, family = binomial(link = "logit"))

# Odds ratios & CIs for model 1
or1  <- exp(coef(lr_result1))
ci1  <- exp(confint(lr_result1))
pval1 <- summary(lr_result1)$coefficients[, "Pr(>|z|)"]

df1 <- data.frame(
  Variable = names(or1),
  OR_new = round(or1, 3),
  LCI1 = round(ci1[, 1], 2),
  UCI1 = round(ci1[, 2], 2),
  P_value1 = round(pval1, 6)
)

# Odds ratios & CIs for model 2
or2  <- exp(coef(lr_result2))
ci2  <- exp(confint(lr_result2))
pval2 <- summary(lr_result2)$coefficients[, "Pr(>|z|)"]

df2 <- data.frame(
  Variable = names(or2),
  OR_all = round(or2, 2),
  LCI2 = round(ci2[, 1], 2),
  UCI2 = round(ci2[, 2], 2),
  P_value2 = round(pval2, 6)
)

# Merge
Result <- merge(df1, df2, by = "Variable", all = TRUE)

#to keep results per dataset in the environment
assign(paste0(Dataset_name, "Result"), Result, envir = .GlobalEnv)

# Write to file
file_to_write <- file.path(output_path_local, paste0(Dataset_name, "Result.csv"))
write.csv(Result, file_to_write, row.names = FALSE)
  
message(paste("Processed:", Dataset_name, "→ Saved to", file_to_write))

}

In [22]:
# call the function to get the results

#Dataset_name <- c("StatinQC","HydrochlorothiazideQC","GabapentinQC","FluticasoneQC") 
#Dataset_name <- c("AmoxicillinQC","MetforminQC","PPIQC","LoopDiureticQC","LosartanQC")
#Dataset_name <- c("AtenololQC", "SildenafilQC")
Dataset_name <- c("PioglitazoneQC", "TNFQC")
temp <- lapply(Dataset_name,regression)


In [14]:
Dataset_name <- c("QuetiapineQC", "SertralineQC", "WarfarinQC", "AcetaminophenQC", "LevothyroxineQC", "LisinoprilQC")
temp <- lapply(Dataset_name,regression)

In [24]:
TNFQCResult

In [16]:
# Add all the dataset to be read
read_file <- function(Dataset_name) {
  
  # Build file path
  tb <- file.path(output_path_local, paste0(Dataset_name,".csv"))
  
  # Read CSV and assign it to an object with the given name
  assign(Dataset_name, read.csv(tb), envir = .GlobalEnv)
}

# Example use
drug_list <- c("QuetiapineQC", "SertralineQC", "WarfarinQC", "AcetaminophenQC", "LevothyroxineQC", "LisinoprilQC")
temp <- lapply(drug_list,read_file)

In [30]:
# Odds ratios & CIs for model 1
or1  <- exp(coef(lr_result1))
ci1  <- exp(confint(lr_result1))
pval1 <- summary(lr_result1)$coefficients[, "Pr(>|z|)"]

df1 <- data.frame(
  Variable = names(or1),
  OR_new = round(or1, 2),
  LCI1 = round(ci1[, 1], 2),
  UCI1 = round(ci1[, 2], 2),
  P_value1 = round(pval1, 3)
)

# Odds ratios & CIs for model 2
or2  <- exp(coef(lr_result2))
ci2  <- exp(confint(lr_result2))
pval2 <- summary(lr_result2)$coefficients[, "Pr(>|z|)"]

df2 <- data.frame(
  Variable = names(or2),
  OR_all = round(or2, 2),
  LCI2 = round(ci2[, 1], 2),
  UCI2 = round(ci2[, 2], 2),
  P_value2 = round(pval2, 3)
)

# Merge
SildenafilQCResult <- merge(df1, df2, by = "Variable", all = TRUE)


# View
#display_df(LosartanResult)

### Analysis Dataset with Personid, Outcome, New User and all user exposure Columns for all drugs

In [26]:
descp <- function(drug_list){
  
  for(i in seq_along(drug_list)){
    
    Dataset_name <- drug_list[i]
    
    # Build file path
    tb <- paste0(output_path_local,Dataset_name,".csv")
    MEDQC <- read.csv(tb)
  
    # Step 1: Prepare analysis dataset
  
    AnalysisData1 <- FinalPatient %>% 
      left_join(ADSL %>% select(PersonId, MCIDT, ADDT, ADT, CNSR, AVAL, Sex, AgeAtDiagnosis, Race, Outcome, Ratio), by = "PersonId") %>%
      left_join(MEDQC %>% select(PersonId, FillStatus, EXSTDT), by = "PersonId") %>%
      left_join(Comorbidity, by = "PersonId") %>%
      mutate(FillStatus = ifelse(is.na(FillStatus),"Unexposed",FillStatus),
           IndexYear = as.numeric(format(as.Date(MCIDT), "%Y"))
     )
  
    # Step 2: Derive exposures
    AnalysisData2 <- AnalysisData1 %>%
      filter(
      Outcome != 2 ) %>%
      mutate(
       Exposure1 = case_when(
        !is.na(EXSTDT) & FillStatus == "Exposed" &
          as.Date(MCIDT) <= as.Date(EXSTDT) &
          as.Date(EXSTDT) <= as.Date(ADT) ~ 1,
        FillStatus == "Unexposed" |
          (!is.na(EXSTDT) & FillStatus == "Exposed" & as.Date(ADT) < as.Date(EXSTDT)) ~ 0,
        TRUE ~ 2
      ),
       Exposure2 = case_when(
        FillStatus == "Exposed" & as.Date(EXSTDT) <= as.Date(ADT) & !is.na(EXSTDT) ~ 1,
        FillStatus == "Unexposed" |
          (FillStatus == "Exposed" & as.Date(ADT) < as.Date(EXSTDT)) ~ 0,
        TRUE ~ 2
      )
    ) %>%
  select(PersonId, Exposure1, Exposure2) %>%
  setNames(c("PersonId",
             paste0(Dataset_name, "_Exposure1"),
             paste0(Dataset_name, "_Exposure2")))
 
    # Step 3: Merge exposures into final
    final <- final %>% left_join(AnalysisData2, by = "PersonId")  
  }
return(final)
}

In [ ]:
# Combine all the drug details together
final <- FinalPatient %>% left_join(ADSL %>% select(PersonId,Outcome),by = "PersonId")
drug_list <- c("StatinQC","HydrochlorothiazideQC","GabapentinQC","FluticasoneQC","AmoxicillinQC","MetforminQC","PPIQC","LoopDiureticQC","LosartanQC","AtenololQC", "SildenafilQC","DexamethasoneQC", "AlbuterolQC", "GLPQC")
AnalysisDataset <- descp(drug_list)
# Write to file
file_to_write <- paste(output_path_local,"AnalysisDataset.csv.r", sep = "")
write.csv(AnalysisDataset, file_to_write, row.names = FALSE)

In [29]:
# If any new drug is ran add it to Analysisdataset
final <- FinalPatient %>% left_join(AnalysisDataset, by = "PersonId")
drug_list <- c("PioglitazoneQC","TNFQC")
AnalysisDataset <- descp(drug_list)
# Write to file
file_to_write <- paste(output_path_local,"AnalysisDataset.csv.r", sep = "")
write.csv(AnalysisDataset, file_to_write, row.names = FALSE)

In [38]:
table(AnalysisDataset$Outcome, AnalysisDataset$TNFQC_Exposure1)

In [29]:
display_df(AnalysisDataset)

In [27]:
AnalysisDataset1 <- AnalysisDataset %>% arrange(PersonId) %>% mutate(subject_id = sprintf("subj%04d", row_number())) %>% select(-PersonId)
 # Write to file
  file_to_write <- paste(output_path_local,"AnalysisDataset1.csv.r", sep = "")
  write.csv(AnalysisDataset1, file_to_write, row.names = FALSE)

#Temporary code; Outcome == 2 not removed

In [13]:
# For the new list of drugs
final <- FinalPatient %>% left_join(ADSL %>% select(PersonId,Outcome),by = "PersonId")
drug_list <- c("AmlodipineQC", "HydralazineQC", "MetoprololQC", "LevothyroxineQC", "LisinoprilQC", "ClopidogrelQC", "CitalopramQC")

AnalysisDataset3 <- descp(drug_list)
# Write to file
file_to_write <- paste(output_path_local,"AnalysisDataset3.csv.r", sep = "")
write.csv(AnalysisDataset3, file_to_write, row.names = FALSE)

In [15]:
MCI_Temp <- FinalPatient %>% 
  left_join(ADSL %>% select(PersonId,ADT, CNSR, AVAL, Sex, AgeAtDiagnosis, Race, IndexYear, Region), by = "PersonId") %>%
  left_join(AnalysisDataset3, by = "PersonId") %>%
  left_join(Comorbidity, by = "PersonId") 

In [17]:
MCI_Temp3 <- MCI_Temp %>% arrange(PersonId) %>% mutate(subject_id = sprintf("subj%04d", row_number())) %>% select(-PersonId) %>% filter(Outcome != 2)

# Write to file
file_to_write <- paste(output_path_local,"MCI_Temp3.csv.r", sep = "")
write.csv(MCI_Temp3, file_to_write, row.names = FALSE)

In [34]:
final <- FinalPatient %>% left_join(ADSL %>% select(PersonId,Outcome),by = "PersonId")
drug_list <- c("SemaglutideQC")
AnalysisDataset <- descp(drug_list)

In [50]:
AnalysisDataset <- AnalysisDataset %>% left_join(temp,by="PersonId")
# Write to file
file_to_write <- paste(output_path_local,"AnalysisDataset.csv.r", sep = "")
write.csv(AnalysisDataset, file_to_write, row.names = FALSE)

In [17]:
# Mutate Index year
ADSL <- ADSL %>% mutate(IndexYear = as.numeric(format(as.Date(MCIDT), "%Y")))
# Write ADSL
file_to_write <- paste(output_path_local, "ADSL.csv.r", sep = "")
write.csv(ADSL, file_to_write, row.names = FALSE)
# Check the outcome and Index year count
#table(Outcome=ADSL$Outcome,Year=ADSL$IndexYear)

In [ ]:
#Patient Characteristics
library(dplyr)
library(tidyr)
library(knitr)

data <- FinalPatient %>% left_join(ADSL,by="PersonId")

summary_table <- function(df, var) {
  df %>%
    count(!!sym(var)) %>%
    mutate(Percentage = round(n / sum(n) * 100, 2)) %>%
    rename(Category = !!sym(var))
}

# Creating summaries for all categorical variables

age_summary <- summary_table(data, "Age_Group")
sex_summary <- summary_table(data, "Sex")
ethnicity_summary <- summary_table(data, "Ethnicity")
race_summary <- summary_table(data, "Race")
marital_summary <- summary_table(data, "MaritalStatus")
#bmi_summary <- summary_table(data, "BMI_Group")
#bmi_impute_summary <- summary_table(data, "BMI_GROUP_impute")
region_summary <- summary_table(data, "Region")
year_summary <- summary_table(data, "IndexYear")
state_summary <- summary_table(data, "StateOrProvince")


# Combine all summaries into one table
final_summary <- bind_rows(
  mutate(age_summary, Variable = "Age_Group"),
  mutate(sex_summary, Variable = "Sex"),
  mutate(ethnicity_summary, Variable = "Ethnicity"),
  mutate(race_summary, Variable = "Race"),
  mutate(marital_summary, Variable = "MaritalStatus"),
  mutate(region_summary, Variable = "Region"),
  mutate(region_summary, Variable = "IndexYear"),
  mutate(state_summary, Variable = "StateOrProvince")
) %>%
  select(Variable, Category, n, Percentage) %>%
  arrange(Variable)

# Display the final summary table
kable(final_summary, caption = "Patient Characteristics Summary for MCI")


In [42]:
# Top medications in the cohort
# Data Driven Approach to check top medication in the cohort

sql <-"

WITH part1 as (
SELECT mcc.CodeConceptId, COUNT(DISTINCT m.PersonId) AS PersonCount
  FROM MedicationRequest m 
  INNER JOIN MedicationCodeConceptMap mcc 
    ON m.CodeConceptMapId = mcc.Id
    GROUP BY mcc.CodeConceptId
)

SELECT  a.CodeConceptId, a.PersonCount, c.ConceptName as MedName
FROM part1 as a
LEFT JOIN Concept c on a.CodeConceptId = c.ConceptId
ORDER BY a.PersonCount DESC
LIMIT 100
"
topDrug <- load_sql_table(con, snapshot, sql, view_name='topDrug',output_mode = "sparklyr") %>% collect()
#display_df(topDrug,10)

In [43]:
display_df(topDrug,10)

Approach 1:
1. Dataset with all Ex and EXQC variables 
2. Divide EX and EXQC
3. Return seperately

In [ ]:
med_flag <- function(codes = "codes", med_name = "name") {
create_view(codes, "med_codes")
  
sql <- "

--EX: De-DUP based on essential variables and select subjects with exposure to DOI
--Only medications needed for analysis
--Filter concept maps and names for the medications wanted
 
WITH map as 
(
    SELECT mcm.Id, cc.*
    FROM  MedicationCodeConceptMap mcm 
    JOIN Concept cc on (mcm.CodeConceptId = cc.ConceptId)
    WHERE mcm.CodeConceptId IN
        (SELECT ConceptId FROM med_codes)
),

med_base as 
(
--Remove error, canceled,declined or stopped dispense
    SELECT *
    FROM MedicationDispense
    WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)
),

--tb1 Join Dispense data with concepts
tb1 as
(
    SELECT DISTINCT d.PersonId, d.DispenseDateTime,d.DaysSupply, 
    ROUND(d.DispenseQuantity, 2) AS DispenseQuantity, mcm.ConceptDefinition as MedicationDisDef, d.DispenseQuantityUOMConceptId
     FROM med_base d
      INNER JOIN map mcm
       ON mcm.Id = d.CodeConceptMapId
),

---tb2 To get the unit name from unit code

tb2 as (
    SELECT  d.*, c.ConceptName as DispenseQuantityUOM 
    FROM tb1 d join Concept c on (d.DispenseQuantityUOMConceptId = c.ConceptId)
),

-- To derive earliest and latest dispense date for each subject for each drug
tb3 as
(
    SELECT PersonId, min(DispenseDateTime) as DSSTDT, max(DispenseDateTime) as DSENDT,
     count(distinct DispenseDateTime) as TotNumDisRecDrug, sum(DaysSupply) as TotalDaysSupply
    FROM tb2
    GROUP BY PersonId
),

 -- Join subject level data, drug related information with tb2
 tb4 as
 (
    SELECT a.*, b.DSSTDT, b.DSENDT, DATEDIFF( b.DSENDT,b.DSSTDT) as DISSDAYSDRUG,
    b.TotNumDisRecDrug, b.TotalDaysSupply
    FROM tb2 a 
    JOIN tb3 b ON a.PersonId = b.PersonId
),

med_base1 as 
(
--Remove error, canceled,declined or stopped dispense
    SELECT *
    FROM MedicationRequest
    WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)
),

--tb1 Join Request data with concepts
tb1r as
(
    SELECT DISTINCT d.PersonId, d.AuthoredOnDateTime as RequestDateTime, mcm.ConceptDefinition as MedicationReqDef
     FROM med_base1 d
      INNER JOIN map mcm
       ON mcm.Id = d.CodeConceptMapId
),

-- To derive earliest and latest request date for each subject for each drug
tb2r as
(
    SELECT PersonId, min(RequestDateTime) as RQSTDT, max(RequestDateTime) as RQENDT,
     count(distinct RequestDateTime) as TotNumReqRecDrug
    FROM tb1r
    GROUP BY PersonId
),

 -- Join subject level data, drug related information with tb2
 tb3r as 
 (
    SELECT a.*, b.RQSTDT, b.RQENDT, DATEDIFF( b.RQENDT,b.RQSTDT) as REQDAYSDRUG,
    b.TotNumReqRecDrug
    FROM tb1r a 
    JOIN tb2r b on a.PersonId = b.PersonId
)

SELECT a.*, b.RequestDateTime,b.MedicationReqDef, b.RQSTDT, b.RQENDT, DATEDIFF( b.RQSTDT,b.RQENDT) as REQDAYSDRUG,
    b.TotNumReqRecDrug, '%s' as DRUGNAME
    FROM tb4 a 
    FULL OUTER JOIN tb3r b on (a.PersonId = b.PersonId)
"
sql1 <- sprintf(sql,med_name)

tb <- load_sql_table(con,snapshot, query = sql1, view_name = 'tb', output_mode = "sparklyr" )

sqlex <- "
      SELECT PersonId, DispenseDateTime, RequestDateTime, DaysSupply, DispenseQuantity, 
      MedicationDisDef, MedicationReqDef, DispenseQuantityUOMConceptId, DispenseQuantityUOM,DRUGNAME
      FROM tb
"
sqlexqc <- "
      SELECT DISTINCT PersonId, DSSTDT, DSENDT, RQSTDT, RQENDT, DISSDAYSDRUG,  REQDAYSDRUG, TotNumDisRecDrug, TotNumReqRecDrug, TotalDaysSupply, DRUGNAME
      FROM tb
"

EX <- load_sql_table(con,snapshot, query = sqlex, output_mode = "sparklyr")
EXQC <- load_sql_table(con,snapshot, query = sqlexqc, output_mode = "sparklyr") %>% collect()

return(list(EX = EX, EXQC = EXQC))

}

Approach 2:

In [ ]:
med_flag <- function(codes = "codes", med_name = "name") {
  create_view(codes, "concept_code")
  
  sql_ex <- "
  WITH tb1 AS (
      SELECT DISTINCT d.PersonId, d.DispenseDateTime, d.CodeConceptId, d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId
      FROM MedicationDispense d
      INNER JOIN MedicationCodeConceptMap mcm ON mcm.Id = d.CodeConceptMapId
      INNER JOIN concept_code cc ON cc.ConceptId = mcm.CodeConceptId
  ),
  tb2 AS (
      SELECT t1.*, c.ConceptName AS ConceptName
      FROM tb1 t1
      JOIN Concept c ON t1.CodeConceptId = c.ConceptId
  ),
  tb3 AS (
      SELECT PersonId, MIN(DispenseDateTime) AS DSSTDT, MAX(DispenseDateTime) AS DSENDT, COUNT(*) AS TotNumDisRecDrug
      FROM tb2
      GROUP BY PersonId
  )

  tb4 AS (
      SELECT PersonId, MIN(DispenseDateTime) AS DispenseDataFirstDT, MAX(DispenseDateTime) AS DispenseDataLastDT, COUNT(*) AS TotNumDisRec
      FROM MedicationDispense
      GROUP BY PersonId
  )

  SELECT d.PersonId, d.DispenseDateTime, d.CodeConceptId, d.ConceptName, d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
         t3.DSSTDT, t3.DSENDT, t3.TotNumDisRecDrug, t4.DispenseDataFirstDT, t4.DispenseDataLastDT, t4.TotNumDisRec, 
         (t4.DispenseDataLastDT - t4.DispenseDataFirstDT) AS DISSDAYS, (t3.DSENDT - t3.DSSTDT) AS DISSDAYSDRUG, '%s' AS TRT
  FROM tb2 d
  LEFT JOIN tb3 t3 ON d.PersonId = t3.PersonId
  LEFT JOIN tb4 t4 ON d.PersonId = t4.PersonId
  "

  sql_ex_fun <- sprintf(sql_ex, med_name)

  ex <- load_sql_table(con, snapshot, query = sql_ex_fun, output_mode = "sparklyr") %>% collect()

}


Return EX and EXQC seperately

In [ ]:
med_flag <- function(codes = "codes", med_name = "name") {
  create_view(codes, "concept_code")
  
  sql_ex <- "
  WITH tb1 AS (
      SELECT DISTINCT d.PersonId, d.DispenseDateTime, d.CodeConceptId, d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId
      FROM MedicationDispense d
      INNER JOIN MedicationCodeConceptMap mcm ON mcm.Id = d.CodeConceptMapId
      INNER JOIN concept_code cc ON cc.ConceptId = mcm.CodeConceptId
  ),
  tb2 AS (
      SELECT t1.*, c.ConceptName AS ConceptName
      FROM tb1 t1
      JOIN Concept c ON t1.CodeConceptId = c.ConceptId
  ),
  tb3 AS (
      SELECT PersonId, MIN(DispenseDateTime) AS DSSTDT, MAX(DispenseDateTime) AS DSENDT, COUNT(*) AS TotNumDisRecDrug
      FROM tb2
      GROUP BY PersonId
  )
  SELECT d.PersonId, d.DispenseDateTime, d.CodeConceptId, d.ConceptName, d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
         t3.DSSTDT, t3.DSENDT
  FROM tb2 d
  LEFT JOIN tb3 t3 ON d.PersonId = t3.PersonId
  "
  
  sql_exqc <- "
  WITH tb1 AS (
      SELECT DISTINCT d.PersonId, d.DispenseDateTime, d.CodeConceptId, d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId
      FROM MedicationDispense d
      INNER JOIN MedicationCodeConceptMap mcm ON mcm.Id = d.CodeConceptMapId
      INNER JOIN concept_code cc ON cc.ConceptId = mcm.CodeConceptId
  ),

  tb2 AS (
      SELECT PersonId, MIN(DispenseDateTime) AS DSSTDT, MAX(DispenseDateTime) AS DSENDT, COUNT(*) AS TotNumDisRecDrug
      FROM tb1
      GROUP BY PersonId
  ),
  tb3 AS (
      SELECT PersonId, MIN(DispenseDateTime) AS DispenseDataFirstDT, MAX(DispenseDateTime) AS DispenseDataLastDT, COUNT(*) AS TotNumDisRec
      FROM MedicationDispense
      GROUP BY PersonId
  )
  SELECT t4.*, 
         (DispenseDataLastDT - DispenseDataFirstDT) AS DISSDAYS,
         (t3.DSENDT - t3.DSSTDT) AS DISSDAYSDRUG,
         t3.TotNumDisRecDrug, '%s' AS TRT
  FROM tb4 t3
  INNER JOIN tb3 t3 ON t4.PersonId = t3.PersonId
  "
  
  sql_exqc_filled <- sprintf(sql_exqc, med_name)

  ex <- load_sql_table(con, snapshot, query = sql_ex, output_mode = "sparklyr") %>% collect()
  exqc <- load_sql_table(con, snapshot, query = sql_exqc_filled, output_mode = "sparklyr") %>% collect()

  return(list(EX = ex, EXQC = exqc))
}


In [ ]:
sql <- "

-- De-DUP based on essential variables and select subjects with exposure to DOI
WITH tb1 as
(
    SELECT DISTINCT d.PersonId, d.DispenseDateTime, d.CodeConceptId, d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
     FROM MedicationDispense d
    INNER JOIN MedicationCodeConceptMap mcm
      ON mcm.Id = d.CodeConceptMapId
    INNER JOIN concept_code cc
     ON cc.ConceptId = mcm.CodeConceptId
),

-- Join with Concept to get the name of ConceptId
tb2 as 
(
  SELECT t1.*, c.ConceptName as ConceptName
   FROM tb1 t1 join Concept c 
    ON p.CodeConceptId = c.ConceptId
),

-- To serive earliest and latest dispense date for each subject for each drug
tb3 as
(
    SELECT PersonId, min(DispenseDateTime) as DSSTDT, max(DispenseDateTime) as DSENDT, count(*) as TotNumDisRecDrug
    FROM tb2
    GROUP BY PersonId
),

-- EX: Merge the table 2 data with table 3 to get the DSSTDT, DSENDT
EX as
(
    SELECT d.PersonId, d.DispenseDateTime, d.CodeConceptId, d.ConceptName, d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
    t3.DSSTDT, t3.DSENDT
    FROM tb2 d LEFT JOIN tb3 as t3
     ON d.PersonId = t3. PersonId
)

-- EXQC: subject level data
tb4 as
(
   SELECT PersonId, min(DispenseDateTime) as DispenseDataFirstDT, max(DispenseDateTime) as DispenseDataLastDT, count(*) as TotNumDisRec
   FROM MedicationDispense
   GROUP BY PersonId
)

EXQC as
(
   SELECT t4.*, (DispenseDataFirstDT -  DispenseDataLastDT) as DISSDAYS, t3.DSSTDT, t3.DSENDT, 
   t3.DSSTDT - t3.DSENDT as DISSDAYSDRUG, t3.TotNumDisRecDrug, %s as TRT
   FROM tb4 t4 INNER JOIN tb3 t3
    ON t4.PersonId=t3.PersonId
)

"

In [39]:
summary(MED_QC1$Ratio)

In [43]:
summary(ADSL1$TotalDiagEnc)

In [ ]:
library(MatchIt)

# Matching
match_obj1 <- matchit(
  NewUExp ~ Sex + AgeAtDiagnosis + Race + IndexYear + IHD + T2DM + Hypertension + Stroke + Depression,
  data = NewUData,
  method = "nearest",
  distance = "glm",
  ratio = 1,
  replace = FALSE,
  caliper = 0.25,         # in SD units
  std.caliper = TRUE,     # explicitly use SD units for caliper
)

# Matching
match_obj2 <- matchit(
  AllUData ~ Sex + AgeAtDiagnosis + Race + IndexYear + IHD + T2DM + Hypertension + Stroke + Depression,
  data = NewUData,
  method = "nearest",
  distance = "glm",
  ratio = 1,
  replace = FALSE,
  caliper = 0.25,         # in SD units
  std.caliper = TRUE,     # explicitly use SD units for caliper
)

#Univariate Logistic regression
matched_data <- match.data(match_obj)
lr_result1 <- glm(Outcome ~ AllUExp, data = matched_data, family = binomial(link = "logit"))
summary(lr_result)

In [ ]:
#Quality Checks
summary(match_obj2)
summary(match_obj1)

In [90]:
# Install if needed
#install.packages("MatchIt")
library(MatchIt)

# Matching
match_obj1 <- matchit(
  NewUExp ~ Sex + AgeAtDiagnosis + Race + IndexYear + IHD + T2DM + Hypertension + Stroke + Depression,
  data = NewUData,
  method = "nearest",
  distance = "glm",
  ratio = 1,
  replace = FALSE,
  caliper = 0.25,         # in SD units
  std.caliper = TRUE,     # explicitly use SD units for caliper
)
# Matching summary
summary(match_obj)

In [94]:
# Install if needed
#install.packages("MatchIt")
library(MatchIt)

# Matching
match_obj <- matchit(
  AllUExp ~ Sex + AgeAtDiagnosis + Race + IndexYear + IHD + T2DM + Hypertension + Stroke + Depression,
  data = AllUData,
  method = "nearest",
  distance = "glm",
  ratio = 1,
  replace = FALSE,
  caliper = 0.25,         # in SD units
  std.caliper = TRUE,     # explicitly use SD units for caliper
)
# Matching summary
summary(match_obj)

In [91]:
#install.packages("cobalt")
library(cobalt)

# Love plot for before/after balancelove.plot(match_obj, binary = "std", var.order = "unadjusted", abs = TRUE)
# Detailed balance table
bal.tab(match_obj, un = TRUE, m.threshold = 0.1) # un=TRUE shows before/after

In [95]:
#install.packages("cobalt")
library(cobalt)

# Love plot for before/after balancelove.plot(match_obj, binary = "std", var.order = "unadjusted", abs = TRUE)
# Detailed balance table
bal.tab(match_obj, un = TRUE, m.threshold = 0.1) # un=TRUE shows before/after

In [96]:
matched_data <- match.data(match_obj)
lr_result <- glm(Outcome ~ AllUExp, data = matched_data, family = binomial(link = "logit"))
summary(lr_result)

In [92]:
matched_data <- match.data(match_obj)
lr_result <- glm(Outcome ~ NewUExp, data = matched_data, family = binomial(link = "logit"))
summary(lr_result)

In [74]:
dataset <- ExposureStatus

cat(sum(dataset$TotNumReqRec == 0 & dataset$TotNumDisRec >= 1, na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRec == 0 & dataset$TotNumDisRecDrug >= 1, na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug != 0 & dataset$TotNumDisRecDrug >= 1, na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug != 0 & ((dataset$TotNumDisRecDrug >= 1 & dataset$TotalDaysSupply >= 30) | dataset$TotNumDisRecDrug >= 2), na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug >= 2 & dataset$TotNumDisRecDrug == 0 & dataset$Ratio < 0.5, na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug >= 2 & dataset$Ratio < 0.5 & (dataset$TotNumDisRecDrug == 0 | (dataset$TotNumDisRecDrug == 1 & dataset$TotalDaysSupply < 30)), na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug >= 2 & dataset$Ratio < 0.25 & (dataset$TotNumDisRecDrug == 0 | (dataset$TotNumDisRecDrug == 1 & dataset$TotalDaysSupply < 30)), na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug >= 2 & dataset$Ratio < 0.75 & (dataset$TotNumDisRecDrug == 0 | (dataset$TotNumDisRecDrug == 1 & dataset$TotalDaysSupply < 30)), na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug == 0 & dataset$TotNumDisRecDrug == 0, na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug >= 2 & dataset$TotNumDisRecDrug == 0 & dataset$Ratio >= 1, na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug >= 1 & dataset$TotNumDisRecDrug == 0 & dataset$Ratio >= 1, na.rm = TRUE), "\n")
cat(sum(dataset$TotNumReqRecDrug >= 1 & dataset$TotNumDisRecDrug == 0 & dataset$Ratio >= 0.75, na.rm = TRUE), "\n")
cat(sum((dataset$TotNumReqRecDrug >= 2 & dataset$TotNumDisRecDrug == 0 & dataset$Ratio >= 0.5 & dataset$Ratio < 1) | 
        (dataset$TotNumReqRecDrug == 1 & dataset$TotNumDisRecDrug == 0), na.rm = TRUE), "\n")


In [77]:

cat(sum(dataset$TotNumReqRecDrug >= 2 & dataset$TotNumDisRecDrug == 0 & dataset$Ratio >= 0.5 & dataset$Ratio < 1,na.rm = TRUE),"\n")
cat(sum(dataset$TotNumReqRecDrug == 1 & dataset$TotNumDisRecDrug == 0, na.rm = TRUE), "\n")

In [26]:
display_df(ADSL1)

In [22]:
ADSL1 <- ADSL %>% 
        mutate(Outcome = case_when(is.na(ADDT) ~ 0,
                         !is.na(ADDT) & ADTotalDiagC >=2 ~ 1,
                         !is.na(ADDT) & ADTotalDiagC ==1 & (NamzaricFl == 1 | antiamyloidFl == 1 | CholinesteraseFl == 1 | MemantineFl == 1) ~ 1,
                         TRUE ~ 2))
#Quality check,
cat("Total AD:",sum(ADSL1$Outcome == 1),"\n")
cat("Total Non-AD:",sum(ADSL1$Outcome == 0),"\n")
cat("Total ND:",sum(ADSL1$Outcome == 2),"\n")

In [17]:
str(MED_QC)

In [52]:
table(AnalysisDataset$SemaglutideQC_Exposure1)

In [53]:
table(AnalysisDataset$SemaglutideQC_Exposure2)